In [41]:
from algorithms.persistance import load_from_json

import os
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt

In [42]:
def convert_to_df(path: str):
    results = load_from_json(path)
    config_list = []
    performance_list = []
    for result in results:
        config_list.append(result['config'])
        performance_list.append(result['performance'])
    config = pd.DataFrame(config_list)
    performance = pd.DataFrame(performance_list)
    result_list = pd.concat([performance, config], axis=1)
    return result_list

In [43]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [44]:
name = 'results'
path = f'../persistent_data/{name}.json'
result_list = convert_to_df(path)
result_list

,true_positives,false_positives,true_negatives,false_negatives,correct_alarm_count,exploit_count,detection_rate,consecutive_false_positives_normal,consecutive_false_positives_exploits,recall,...,lids version,scenario,ngram,embedding,algorithm,window,detection_time,flag,mode,process_name
0,0,171,6343324,1032644,0,139,0.000000,8,4,0.000000,...,2021,Bruteforce_CWE-307/,5,int,stide,100,6.565811,True,True,True
1,3061438,1479,6411589,9260439,114,120,0.950000,1,2,0.950000,...,2021,CVE-2012-2122/,3,int,stide,300,7.478978,True,True,True
2,7496,1185,6342310,1025148,50,139,0.359712,9,1,0.359712,...,2021,Bruteforce_CWE-307/,5,int,stide,300,6.503134,True,True,True
3,393669,571,6412497,11928208,81,120,0.675000,0,4,0.675000,...,2021,CVE-2012-2122/,3,int,stide,100,7.607554,True,True,True
4,3894841,2168,6410900,8427036,119,120,0.991667,2,2,0.991667,...,2021,CVE-2012-2122/,3,int,stide,500,7.649343,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,3065298,961,6410343,9256579,108,120,0.900000,3,1,0.900000,...,2021,CVE-2012-2122/,5,int,stide,500,7.586096,True,True,False
86,20355,8959,25340417,788965,120,120,1.000000,134,1,1.000000,...,2021,CVE-2020-23839/,3,int,stide,100,10.269251,True,True,False
87,52840,37176,25312200,756480,120,120,1.000000,105,8,1.000000,...,2021,CVE-2020-23839/,3,int,stide,500,9.834013,True,True,False
88,88561,55973,25293403,720759,120,120,1.000000,77,1,1.000000,...,2021,CVE-2020-23839/,3,int,stide,1000,9.900332,True,True,False


In [45]:
conn = sql.connect(f'{name}.db')
try:
    result_list.to_sql(name, conn)
except ValueError:
    os.remove(f'{name}.db')
    print('Table already exists - remove tables')

/home/tk-whk/Documents/Environment/lids/lib/python3.9/site-packages/pandas/core/generic.py:2605: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [46]:
res = conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in res.fetchall():
    print(name[0])
name = name[0]

results


In [47]:
result = pd.read_sql(f'SELECT "lids version", \
                              algorithm, \
                              ngram, \
                              window, \
                              process_name, \
                              AVG(detection_rate) as DR, \
                              AVG(consecutive_false_positives_exploits)+ AVG(consecutive_false_positives_normal) as CFP_sum, \
                              Count(*), \
                              AVG(detection_time) as avg_time_in_min \
                              FROM  {name} \
                     GROUP BY ngram, window, process_name, "lids version"\
                     ORDER BY DR desc, \
                              CFP_sum asc', conn)
pd.set_option('display.max_rows', result.shape[0]+1)
result

,lids version,algorithm,ngram,window,process_name,DR,CFP_sum,Count(*),avg_time_in_min
0,2021,stide,5,700,0,1.000000,2.000000,1,0.532374
1,2021,stide,3,1000,1,1.000000,52.500000,4,20.844454
2,2021,stide,3,700,1,1.000000,59.000000,4,21.137337
3,2021,stide,3,500,1,0.997917,65.250000,4,21.035332
4,2021,stide,3,300,1,0.987500,83.250000,4,20.840694
5,2021,stide,3,100,1,0.898258,124.750000,4,21.092342
6,2021,stide,3,1000,0,0.799245,23.200000,5,4.161418
7,2021,stide,5,500,1,0.752778,21.000000,3,12.349225
8,2021,stide,5,700,1,0.747222,42.333333,3,12.062350
9,2021,stide,5,500,0,0.731250,14.250000,4,3.772257
